In [224]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from plotly.subplots import make_subplots

In [225]:
nvda = yf.Ticker("NVDA")
amd = yf.Ticker("amd")
asml = yf.Ticker("asml")
sp500 = yf.Ticker("^GSPC")
stoxx50 = yf.Ticker("^STOXX50E")

# get historical market data
df = asml.history(period="1y", interval="1d")

fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close']
                                     )])

begin_date, end_date = [df.iloc[0].name, df.iloc[-1].name]
df = df.reindex(pd.date_range(begin_date, end_date, freq='D'))
datebreaks = df['Close'][df['Close'].isnull()].index
fig.update_xaxes(rangebreaks=[dict(values=datebreaks)])

fig.update_yaxes(title_text="y-axis in logarithmic scale", type="log")

fig.show()


In [226]:
available_closedata = df['Close'][~df['Close'].isnull()]

dr = []

#TODO: Make matrix operation
for i, close_price in enumerate(available_closedata[1:]):
    dr += [close_price/available_closedata[i]-1]

print(np.mean(dr))
print((1+np.mean(dr))**(len(available_closedata))-1)
print(np.std(dr))

0.002128631978506459
0.712529507711533
0.02018360276772606


C:\Users\Julian\AppData\Local\Temp\ipykernel_15252\3273108272.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [227]:
dist1 = sp.stats.t
res1 = sp.stats.fit(dist1, dr, bounds = [(0,len(dr)),(-0.1,0.1), (0,0.1)])
dist2 = sp.stats.norm
res2 = sp.stats.fit(dist2, dr, bounds = [(-0.1,0.1), (0,0.1)])
res = res1
print(res)
print(res.nllf())

fig1 = px.histogram(dr, nbins=20, marginal="box", histnorm="probability density")
fig2 = px.line(x=np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), y=dist.pdf(np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), *res.params))
fig2.update_traces(line_color='red')

fig = make_subplots(rows=2, cols=1, shared_yaxes='all', specs=[[{'type': 'box'}], [{'type': 'histogram'}]])

fig.add_trace(go.Box(fig1.data[1]), row=1, col=1)
fig.add_trace(go.Histogram(fig1.data[0]), row=2, col=1)
fig.add_trace(go.Scatter(fig2.data[0]), secondary_y=False, row=2, col=1)

fig.update_layout(yaxis=dict(domain=[0.8516, 1.0]),
                  yaxis2=dict(domain=[0.0, 0.8316],),
                  xaxis=dict(showticklabels=False),                  
                 )

fig.show()

c:\Users\Julian\miniconda3\envs\financialProductAnalysis\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2357: RuntimeWarning:

overflow encountered in divide



  params: FitParams(df=6.558896586850552, loc=0.0020317349767908716, scale=0.016789423212124118)
 success: True
 message: 'Optimization terminated successfully.'
-631.4797094176647


In [228]:
print(dist1(*res1.params).mean())
def risk(dist, res, risk_free_rate=0.0):
    #TODO: integration by parts
    return sp.integrate.quad(lambda x: x*dist(*res.params).pdf(x), -np.inf, 0)[0]

print(risk(dist1, res1))
print(risk(dist1, res1)/ dist1(*res1.params).mean())






0.0020317349767908716
-0.006639743512507875
-3.2680165417023828
